# COMP551 Mini Project 2 - IMDB Sentiment Analysis  
This is the codes of mini project2 - IMDB Sentiment Analysis.  

## AUTHORS
Pengnan Fan, ID#260768510  

## TASKS
1. **Bernoulli Naive Bayes** (w/o any external library).  
2. **At least 2** out of 3 classifiers from the SciKit. i.e. suggestions: logistic regression, decision tree, or support vector machines  
3. **At least 2** different features extraction pipelines for processing the data.  
4. A model validation. i.e. **K-fold cross validation**  

## UPDATES
**<February 6, 2019>** Pengnan Fan create this notebook and implements functions readTrainData and readTestData  
> * ~~**readTrainData(address:String):DataFrame**~~  
> ~~This function takes a string **address** which indicates the address of your train data and will load comments and isPositive to a DataFrame.~~  
> * ~~**readTestData(address:String):DataFrame**~~  
> ~~This function takes a string **address** which indicates the address of your test data and will load comments and isPositive to a DataFrame. **Note: all isPositive is initialized as 0**~~  
> * **Learning set** 25000 in total
> * **Test set** 25000 in total  

**<February 7, 2019>** Pengnan Fan implements wordsFrequencyNaive and wordsFrequencyStopword
> * **wordsFrequencyNaive(dataSet:DataFrame)**  
> This function takes a DataFrame **dataSet** and calculate the naive word frequency  
> * **wordsFrequencyStopword(dataSet:DataFrame)**  
> This function takes a DataFrame **dataSet** and calculate the word frequency without stopwords  

**<February 9, 2019>** Pengnan Fan uses sklearn.datasets.load_files(address:str) to load all datas. It helps to increase the speed of loading. And the implementation of Bernoulli naive Bayes has been started.  
> * **sklearn.datasets.load_files** - [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html)

**<February 10, 2019>** Pengnan Fan implements numOfExistanceNaive and numOfExistanceStopword
> * **numOfExistanceNaive(dataset:Bunch)**  
> This function takes a Bunch **dataSet** and calculate the naive existance of words~~  
> * **numOfExistanceStopword(dataset:Bunch)**  
> This funciton takes a Bunch **dataSet** and calculate the existance of words without stopwords  

**<February 11, 2019>** Pengnan Fan implements bernoulliNaiveBayes and evaluation. Also, the dataset is changed by using loadData  
> * ~~**bernoulliNaiveBayes(dataSet:Bunch, totalWordFreq:set, negWordFreq:set, posWordFreq:set, numOfExamples:list)**~~  
> ~~This function takes a Bunch **dataSet** as learning set and uses three sets of **totalWordFreq**, **negWordFreq**, **posWordFreq** and a list **numOfExamples** to calculates related probabilities. Note: It is not completely corrent. Fix later today.~~  
> ~~* **evaluation(dataSet:Bunch, prediction:list)**  
> This function takes a Bunch **dataSet** and a list **prediction** to generate a set containing true pos\true neg\false pos\false neg
> * **loadData(address:str)**~~  
> This function takes a string **address** and generate a dict of 3 lists (pos, neg, all) of dict ('comment', 'isPos')  

**<February 13, 2019>** Pengnan Fan fixs numOfExistanceStopword and bernoulliNaiveBayes
> * **advancedNumOfExistance(dataset:dict of list of dict)**  
> This funciton takes a dict of list of dict **dataSet** and calculate the existance of words without stopwords, puctuations, and duplicates.  
> * **bernoulliNaiveBayes(dataSet:list of dict, wordExistance:dict of list, size:dict of list)**  
> This function takes a list of dict **dataSet** as predicting set and uses a dict of list **wordExistance** and a dict of list **size** to calculates related probabilities.  
> * **evaluation(dataSet:list of dict, prediction:list)**  
> This function takes a list of dict **dataSet** and a list **prediction** to generate a set containing true pos\true neg\false pos\false neg  
> * **Outcome** TP =  11848, TN =  11482, FP =  1018, FN =  652  
>> accuracy on train set =  93.32 %  
>> accuracy on test set = 50%  
> **Possible Reason** Overfitting caused by too many features.  

**<February 18, 2019>** Pengnan finished crossvalidation

In [1]:
import pandas as pd
import glob
import sys
import math
from time import sleep
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import nltk
import numpy
import scipy
import sklearn.datasets
import contractions
from itertools import groupby
import string

## Task 0 - Preprocessing Data

### Task 0.1 - Data Loading

In [2]:
# Task 0 - Preprocessing Data

# Please add your address here as string
ADDRESS_TRAIN_PENGNAN = "D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification\\train"
# ADDRESS_TEST_PENGNAN = "D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification\\test"



# @author Pengnan Fan
# @param address of train set
# @return a dict of list of dict
def loadData(address):
    print("Start loading negative set")
    neg = sklearn.datasets.load_files(address, categories={"neg"})
    print("Complete loading negative set")
    print("Start loading positive set")
    pos = sklearn.datasets.load_files(address, categories={"pos"})
    print("Complete loading postive set")
    
    negSet = list()
    count = 0
    size = len(neg.data)
    for x in neg.data:
        negSet.append({"comment":x.decode('utf-8'), "isPos":0})
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(size)) * 100)
        sys.stdout.write("Preparing negative set: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    print()
    
    posSet = list()
    count = 0
    size = len(pos.data)
    for x in pos.data:
        posSet.append({"comment":x.decode('utf-8'), "isPos":1})
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(size)) * 100)
        sys.stdout.write("Prepare positive set: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    output = {'pos':posSet, 'neg':negSet, 'all':posSet+negSet}
    print("\nFinish preparing")
    return output

# Advanced loader
# @return: Bunch

trainSet = loadData(ADDRESS_TRAIN_PENGNAN)

Start loading negative set
Complete loading negative set
Start loading positive set
Complete loading postive set
Preparing negative set: [=================== ] 99%
Prepare positive set: [=================== ] 99%
Finish preparing


# Task 0.2 - Naive Word Frequency

In [3]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: naive word frequency
def wordsFrequencyNaive(dataSet):
    naiveCount = dict()
    totalComments = []
    count = 0
    lenW = len(dataSet['pos'])
    
    print("Start counting naive word frequency of positive set")
    for comment in dataSet['pos']:
        totalComments+=comment['comment'].lower().split(" ")
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete counting naive word frequency of positive set")
    
    posCount = Counter(x for x in totalComments)
    
    totalComments = []
    count = 0
    lenW = len(dataSet['neg'])
    
    print("Start counting naive word frequency of negative set")
    for comment in dataSet['neg']:
        totalComments+=comment['comment'].lower().split(" ")
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete counting naive word frequency of negative set")
    
    negCount = Counter(x for x in totalComments)
    
    totalComments = []
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting naive word frequency of all set")
    for comment in dataSet['all']:
        totalComments+=comment['comment'].lower().split(" ")
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete counting naive word frequency of all set")
    
    allCount = Counter(x for x in totalComments)
    
    naiveCount = {'pos':posCount, 'neg':negCount, 'all':allCount}
    return naiveCount


naiveFrequency = wordsFrequencyNaive(trainSet)

Start counting naive word frequency of positive set
[=================== ] 99%
Complete counting naive word frequency of positive set
Start counting naive word frequency of negative set
[=================== ] 99%
Complete counting naive word frequency of negative set
Start counting naive word frequency of all set
[=================== ] 99%
Complete counting naive word frequency of all set


In [4]:
# An example of naive word frequency
'''
print(naiveFrequency.most_common(160))
print("Size of naiveFrequency: ", len(naiveFrequency))
print("Example: the = ", naiveFrequency['the'])
print(naiveFrequency_neg.most_common(160))
print("Size of naiveFrequency_neg: ", len(naiveFrequency_neg))
print("Example: the = ", naiveFrequency_neg['the'])
print(naiveFrequency_pos.most_common(160))
print("Size of naiveFrequency_pos:", len(naiveFrequency_pos))
print("Example: the = ", naiveFrequency_pos['the'])
'''
# print(naiveFrequency['all'].most_common(160))

'\nprint(naiveFrequency.most_common(160))\nprint("Size of naiveFrequency: ", len(naiveFrequency))\nprint("Example: the = ", naiveFrequency[\'the\'])\nprint(naiveFrequency_neg.most_common(160))\nprint("Size of naiveFrequency_neg: ", len(naiveFrequency_neg))\nprint("Example: the = ", naiveFrequency_neg[\'the\'])\nprint(naiveFrequency_pos.most_common(160))\nprint("Size of naiveFrequency_pos:", len(naiveFrequency_pos))\nprint("Example: the = ", naiveFrequency_pos[\'the\'])\n'

### Task 0.3 - Word Frequency without Stopword

In [5]:
# @author Pengnan Fan
# @acknowledgement Yuxiang Ma, for this function is edited based on his in miniproject1
# @param dataSet: set of comments
# @return naiveCount: word frequency without stopwords
def wordsFrequencyStopword(dataSet):
    stopwordCount = dict()
    totalString = str()
    count = 0
    lenW = len(dataSet['pos'])
    
    print("Start counting naive word frequency of positive set")
    
    for comment in dataSet['pos']:
        totalString = totalString + ' ' + comment['comment'].lower()
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    tokenizer = RegexpTokenizer(r'\w+')
    withoutPunc = tokenizer.tokenize(totalString)
    stopwordsSet = set(stopwords.words())
    posCountDict = Counter(s.lower() for s in withoutPunc if s.lower() not in stopwordsSet)
    print("\nComplete counting naive word frequency of positive set")
    
    totalString = str()
    count = 0
    lenW = len(dataSet['neg'])
    
    print("Start counting naive word frequency of negative set")
    
    for comment in dataSet['neg']:
        totalString = totalString + ' ' + comment['comment'].lower()
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    tokenizer = RegexpTokenizer(r'\w+')
    withoutPunc = tokenizer.tokenize(totalString)
    stopwordsSet = set(stopwords.words())
    negCountDict = Counter(s.lower() for s in withoutPunc if s.lower() not in stopwordsSet)
    print("\nComplete counting naive word frequency of negative set")
    
    totalString = str()
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting naive word frequency of all set")
    
    for comment in dataSet['all']:
        totalString = totalString + ' ' + comment['comment'].lower()
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    tokenizer = RegexpTokenizer(r'\w+')
    withoutPunc = tokenizer.tokenize(totalString)
    stopwordsSet = set(stopwords.words())
    allCountDict = Counter(s.lower() for s in withoutPunc if s.lower() not in stopwordsSet)
    print("\nComplete counting naive word frequency of all set")
    
    stopwordCount = {'pos':posCountDict, 'neg':negCountDict, 'all':allCountDict}
    
    return stopwordCount

# stopwordFrequency = wordsFrequencyStopword(trainSet)

In [6]:
# An example of word frequency without stopwords
'''
print(stopwordFrequency.most_common(160))
print("Size of stopwordFrequency: ", len(stopwordFrequency))
print("Example: br = ", stopwordFrequency['br'])
print(stopwordFrequency_neg.most_common(160))
print("Size of stopwordFrequency_neg: ", len(stopwordFrequency_neg))
print("Example: movie = ", stopwordFrequency_neg['movie'])
print(stopwordFrequency_pos.most_common(160))
print("Size of stopwordFrequency_pos: ", len(stopwordFrequency_pos))
print("Example: movie = ", stopwordFrequency_pos['film'])
'''
# print(stopwordFrequency['all'].most_common(160))

'\nprint(stopwordFrequency.most_common(160))\nprint("Size of stopwordFrequency: ", len(stopwordFrequency))\nprint("Example: br = ", stopwordFrequency[\'br\'])\nprint(stopwordFrequency_neg.most_common(160))\nprint("Size of stopwordFrequency_neg: ", len(stopwordFrequency_neg))\nprint("Example: movie = ", stopwordFrequency_neg[\'movie\'])\nprint(stopwordFrequency_pos.most_common(160))\nprint("Size of stopwordFrequency_pos: ", len(stopwordFrequency_pos))\nprint("Example: movie = ", stopwordFrequency_pos[\'film\'])\n'

### Task 0.4 - Number of Naive Existance of Words

In [7]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: list of words of num of naive existances
def numOfExistanceNaive(dataSet):
    naiveCount = dict()
    totalComments = []
    count = 0
    lenW = len(dataSet['pos'])
    
    print("Start counting number of naive word existance of positive set")
    for comment in dataSet['pos']:
        commentSplit = comment['comment'].lower().split(" ")
        wordsToAdd = []
        for word in commentSplit:
            if word not in wordsToAdd:
                wordsToAdd.append(word)
        totalComments+=wordsToAdd
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    posCount = Counter(x for x in totalComments)
    print("\nComplete counting number of naive word existance of positive set")
    
    totalComments = []
    count = 0
    lenW = len(dataSet['neg'])
    
    print("Start counting number of naive word existance of negative set")
    for comment in dataSet['neg']:
        commentSplit = comment['comment'].lower().split(" ")
        wordsToAdd = []
        for word in commentSplit:
            if word not in wordsToAdd:
                wordsToAdd.append(word)
        totalComments+=wordsToAdd
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    negCount = Counter(x for x in totalComments)
    print("\nComplete counting number of naive word existance of negative set")
    
    totalComments = []
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting number of naive word existance of all set")
    for comment in dataSet['all']:
        commentSplit = comment['comment'].lower().split(" ")
        wordsToAdd = []
        for word in commentSplit:
            if word not in wordsToAdd:
                wordsToAdd.append(word)
        totalComments+=wordsToAdd
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    allCount = Counter(x for x in totalComments)
    print("\nComplete counting number of naive word existance of all set")
    
    naiveCount = {'pos':posCount, 'neg':negCount, 'all':allCount}
    
    return naiveCount

# naiveNumOfExistance = numOfExistanceNaive(trainSet)

In [8]:
# An example of number of naive existance of words
'''
print(naiveNumOfExistance.most_common(160))
print("Size of naiveNumOfExistance: ", len(naiveNumOfExistance))
print("Example: the = ", naiveNumOfExistance['the'])
print(naiveNumOfExistance_neg.most_common(160))
print("Size of naiveNumOfExistance_neg: ", len(naiveNumOfExistance_neg))
print("Example: the = ", naiveNumOfExistance_neg['the'])
print(naiveNumOfExistance_pos.most_common(160))
print("Size of naiveNumOfExistance_pos: ", len(naiveNumOfExistance_pos))
print("Example: the = ", naiveNumOfExistance_pos['the'])
'''
# print(naiveNumOfExistance['pos'].most_common(160))

'\nprint(naiveNumOfExistance.most_common(160))\nprint("Size of naiveNumOfExistance: ", len(naiveNumOfExistance))\nprint("Example: the = ", naiveNumOfExistance[\'the\'])\nprint(naiveNumOfExistance_neg.most_common(160))\nprint("Size of naiveNumOfExistance_neg: ", len(naiveNumOfExistance_neg))\nprint("Example: the = ", naiveNumOfExistance_neg[\'the\'])\nprint(naiveNumOfExistance_pos.most_common(160))\nprint("Size of naiveNumOfExistance_pos: ", len(naiveNumOfExistance_pos))\nprint("Example: the = ", naiveNumOfExistance_pos[\'the\'])\n'

### Task 0.5.1 - Number of Existance of Words without Stopwords and duplicates

In [9]:
# @author Pengnan Fan
# @param dataSet: set of comments
# @return naiveCount: list of num of existances of words without stopwords
def advancedNumOfExistance(dataSet):
    countDict = dict()
    stopwordsSet = set(stopwords.words())
    stopwordsSet.add('br')
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['pos'])
    
    print("Start counting number of words without stopwords of existance of positive set")
    for exp in dataSet['pos']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        # processedComment = ([k for k, v in groupby(sent_clean.split())])
        processedComment = []
        
        for word in sent_clean.split():
            if word not in processedComment:
                processedComment.append(word)
                
        wordSet+=processedComment
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    posDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of positive set")
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['neg'])
    
    print("Start counting number of words without stopwords of existance of negative set")
    for exp in dataSet['neg']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        # processedComment = ([k for k, v in groupby(sent_clean.split())])
        processedComment = []
        
        for word in sent_clean.split():
            if word not in processedComment:
                processedComment.append(word)
                
        wordSet+=processedComment
            
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    negDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of negative set")
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting number of words without stopwords of existance of all set")
    for exp in dataSet['all']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        # processedComment = ([k for k, v in groupby(sent_clean.split())])
        processedComment = []
        
        for word in sent_clean.split():
            if word not in processedComment:
                processedComment.append(word)
                
        wordSet+=processedComment
            
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1     
    
    allDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of all set")
    
    countDict = {'neg':negDict, 'pos':posDict, 'all':allDict}
    
    return countDict


advancedNumOfExistance = advancedNumOfExistance(trainSet)

Start counting number of words without stopwords of existance of positive set
[=================== ] 99%
Complete counting number of words without stopwords of existance of positive set
Start counting number of words without stopwords of existance of negative set
[=================== ] 99%
Complete counting number of words without stopwords of existance of negative set
Start counting number of words without stopwords of existance of all set
[=================== ] 99%
Complete counting number of words without stopwords of existance of all set


In [10]:
# An example of number of existance of words without stopwords
'''
print(stopwordNumOfExistance.most_common(160))
print("Size of stopwordNumOfExistance: ", len(stopwordNumOfExistance))
print("Example: br = ", stopwordNumOfExistance['br'])
print(naiveNumOfExistance_neg.most_common(160))
print("Size of stopwordNumOfExistance_neg: ", len(stopwordNumOfExistance_neg))
print("Example: movie = ", stopwordNumOfExistance_neg['movie'])
print(naiveNumOfExistance_pos.most_common(160))
print("Size of stopwordNumOfExistance_pos: ", len(stopwordNumOfExistance_pos))
print("Example: movie = ", stopwordNumOfExistance_pos['film'])
'''
print(advancedNumOfExistance['neg'].most_common(160)) 
print(advancedNumOfExistance['all'].most_common(160))

[('movie', 8102), ('one', 6817), ('film', 6647), ('like', 6168), ('would', 5325), ('even', 4905), ('good', 4762), ('bad', 4284), ('time', 4138), ('could', 4051), ('really', 4030), ('see', 3857), ('much', 3668), ('get', 3578), ('make', 3435), ('story', 3323), ('made', 3293), ('acting', 3240), ('people', 3133), ('plot', 3065), ('first', 3030), ('movies', 2909), ('well', 2900), ('way', 2892), ('think', 2771), ('watch', 2727), ('characters', 2639), ('better', 2637), ('seen', 2566), ('ever', 2508), ('know', 2495), ('never', 2468), ('say', 2393), ('little', 2333), ('films', 2333), ('cannot', 2318), ('nothing', 2299), ('thing', 2272), ('two', 2268), ('character', 2265), ('something', 2257), ('many', 2241), ('watching', 2152), ('go', 2138), ('great', 2060), ('scenes', 2027), ('worst', 2019), ('going', 1987), ('actually', 1968), ('actors', 1932), ('scene', 1905), ('another', 1872), ('back', 1868), ('still', 1862), ('look', 1819), ('life', 1720), ('best', 1703), ('got', 1699), ('least', 1694), (

### Task 0.5.2 - Without Eliminate Duplicates

In [11]:
def advancedNumOfExistanceWithDuplicates(dataSet):
    output = dict()
    countDict = dict()
    stopwordsSet = set(stopwords.words())
    stopwordsSet.add('br')
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['pos'])
    eachFreqPos = list()
    
    print("Start counting number of words without stopwords of existance of positive set")
    for exp in dataSet['pos']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        # processedComment = ([k for k, v in groupby(sent_clean.split())])
        
        wordSet+=sent_clean.split()
        
        temp_pos = Counter(s for s in sent_clean.split() if s not in stopwordsSet)
        
        eachFreqPos.append(temp_pos)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    posDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of positive set")
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['neg'])
    eachFreqNeg = list()
    
    print("Start counting number of words without stopwords of existance of negative set")
    for exp in dataSet['neg']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        # processedComment = ([k for k, v in groupby(sent_clean.split())])
                
        wordSet+=sent_clean.split()
        
        temp_neg = Counter(s for s in sent_clean.split() if s not in stopwordsSet)
        
        eachFreqNeg.append(temp_neg)
            
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    negDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of negative set")
    
    wordSet = list()
    count = 0
    lenW = len(dataSet['all'])
    
    print("Start counting number of words without stopwords of existance of all set")
    for exp in dataSet['all']: 
        comment = contractions.fix(exp['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        # processedComment = ([k for k, v in groupby(sent_clean.split())])
        
        wordSet+=sent_clean.split()
            
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1     
    
    allDict = Counter(s for s in wordSet if s not in stopwordsSet)
    
    print("\nComplete counting number of words without stopwords of existance of all set")
    
    countDict = {'neg':negDict, 'pos':posDict, 'all':allDict}
    
    eachFreq = eachFreqPos+eachFreqNeg
    
    output = {'allFreq':countDict, 'individual':eachFreq}
    
    return output

wordsWithDuplicates = advancedNumOfExistanceWithDuplicates(trainSet)

Start counting number of words without stopwords of existance of positive set
[=================== ] 99%
Complete counting number of words without stopwords of existance of positive set
Start counting number of words without stopwords of existance of negative set
[=================== ] 99%
Complete counting number of words without stopwords of existance of negative set
Start counting number of words without stopwords of existance of all set
[=================== ] 99%
Complete counting number of words without stopwords of existance of all set


In [12]:
print(wordsWithDuplicates['allFreq']['all'].most_common(100))
print(wordsWithDuplicates['individual'][0])

[('movie', 41809), ('film', 37455), ('one', 25508), ('like', 19641), ('would', 15830), ('good', 14555), ('even', 12503), ('time', 11779), ('really', 11661), ('story', 11454), ('see', 11223), ('much', 9584), ('could', 9381), ('well', 9264), ('get', 9212), ('people', 8951), ('bad', 8912), ('great', 8894), ('first', 8857), ('made', 7990), ('movies', 7788), ('make', 7733), ('films', 7727), ('way', 7685), ('characters', 7291), ('think', 7229), ('watch', 6777), ('two', 6643), ('many', 6640), ('seen', 6530), ('character', 6514), ('never', 6425), ('little', 6387), ('acting', 6291), ('plot', 6275), ('best', 6263), ('love', 6214), ('show', 6083), ('know', 6038), ('life', 5988), ('ever', 5804), ('still', 5561), ('better', 5547), ('say', 5331), ('scene', 5169), ('scenes', 5063), ('go', 4991), ('something', 4892), ('cannot', 4811), ('back', 4760), ('us', 4724), ('watching', 4511), ('real', 4498), ('years', 4456), ('though', 4415), ('thing', 4389), ('actors', 4376), ('another', 4265), ('going', 4262

### Task 0.6 - Data Standardization

In [13]:
# @author Pengnan Fan
# @param dataSet
# @return proceedData
def dataStandardization(dataSet):
    proceedData = list()
    stopwordsSet = set(stopwords.words())
    stopwordsSet.add('br')
    
    count = 0
    lenW = len(dataSet)
    
    print("Start proceeding data")
    for data in dataSet: 
        comment = contractions.fix(data['comment'].lower())
        sent_map = comment.maketrans(dict.fromkeys(string.punctuation))
        sent_clean = comment.translate(sent_map)
        processedComment = sent_clean.split()
        toAdd = list()
        for x in processedComment:
            if x not in stopwordsSet and x not in toAdd:
                toAdd.append(x)
                
        proceedData.append(toAdd)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete proceeding data")
    return proceedData

proceedData_All = dataStandardization(trainSet['all'])

Start proceeding data
[=================== ] 99%
Complete proceeding data


In [14]:
print(proceedData_All[0])
print()
print(trainSet['all'][0]['comment'])

['perhaps', 'arrange', 'meet', 'sitting', 'office', 'doubt', 'would', 'right', 'conversation', 'facetoface', 'bourne', 'remains', 'street', 'tough', 'elusive', 'inhuman', 'resilience', 'leads', 'film', 'little', 'far', 'fantasy', 'macho', 'point', 'shards', 'bond', 'type', 'gallows', 'humour', 'actionbr', 'feeling', 'something', 'going', 'another', 'level', 'world', 'live', 'trilogy', 'coveys', 'well', 'scene', 'set', 'waterloo', 'guardian', 'journalist', 'great', 'effect', 'meeting', 'worlds', 'superfluous', 'bodybr', 'shaky', 'annoy', 'much', 'enjoy', 'hope', 'somehow', 'keep', 'franchise']

"Perhaps we can arrange a meet. " "Where are you now? " "I'm sitting in my office. " "I doubt that. " "Why would you doubt that? " "If you were in your office right now we'd be having this conversation face-to-face. "<br /><br />Bourne remains street tough, and elusive. Only his inhuman resilience leads the film a little too far into fantasy. Conversation is macho, to the point with only shards o

### Task 0.7.1 - CountAllWords

In [15]:
def allWords(dataSet):
    output = list()
    lenW = len(dataSet)
    count = 0
    print("Start counting all words")
    for comments in dataSet:
        for word in comments:
            if word not in output:
                output.append(word)
                
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete counting all words")
    return output

allWord = allWords(proceedData_All)

Start counting all words
[=================== ] 99%
Complete counting all words


In [16]:
print(allWord[0:100])

['perhaps', 'arrange', 'meet', 'sitting', 'office', 'doubt', 'would', 'right', 'conversation', 'facetoface', 'bourne', 'remains', 'street', 'tough', 'elusive', 'inhuman', 'resilience', 'leads', 'film', 'little', 'far', 'fantasy', 'macho', 'point', 'shards', 'bond', 'type', 'gallows', 'humour', 'actionbr', 'feeling', 'something', 'going', 'another', 'level', 'world', 'live', 'trilogy', 'coveys', 'well', 'scene', 'set', 'waterloo', 'guardian', 'journalist', 'great', 'effect', 'meeting', 'worlds', 'superfluous', 'bodybr', 'shaky', 'annoy', 'much', 'enjoy', 'hope', 'somehow', 'keep', 'franchise', 'movie', 'horrible', 'bad', 'good', 'kind', 'waybr', 'storyline', 'rehashed', 'many', 'films', 'even', 'bother', 'describing', 'swordsorcery', 'picture', 'kid', 'hoping', 'realize', 'important', 'nomadic', 'adventurer', 'evil', 'aidesorcerer', 'princess', 'hairy', 'creatureyou', 'get', 'pointbr', 'first', 'time', 'caught', 'harsh', 'winter', 'know', 'decided', 'continue', 'watching', 'extra', 'fiv

### Task 0.7.2 - Advanced CountAllWords (Select Words by IG)

In [17]:
# @author Pengnan Fan
# This function reduce features by information gain
def selectWords(uniqueWordSet, wordExistance, sizeOfSet):
    output = list()
    lenW = len(uniqueWordSet)
    count = 0
    h_D = -(sizeOfSet['pos']/sizeOfSet['all'])*numpy.log2(sizeOfSet['pos']/sizeOfSet['all']) - (sizeOfSet['neg']/sizeOfSet['all'])*numpy.log2(sizeOfSet['neg']/sizeOfSet['all'])
    print("Start selecting words by information gain")
    for word in uniqueWordSet:
        size_with_word = wordExistance['pos'][word] + wordExistance['neg'][word]
        rate_with_word = size_with_word/sizeOfSet['all']
        
        size_without_word = sizeOfSet['all'] - size_with_word
        rate_without_word = 1 - rate_with_word
        
        h_C_with_word = rate_with_word*(-(wordExistance['pos'][word]/size_with_word)*numpy.log2(wordExistance['pos'][word]/size_with_word) - (wordExistance['neg'][word]/size_with_word)*numpy.log2(wordExistance['neg'][word]/size_with_word))
        h_C_without_word = rate_without_word*(-((sizeOfSet['pos']-wordExistance['pos'][word])/size_without_word)*numpy.log2((sizeOfSet['pos']-wordExistance['pos'][word])/size_without_word) - ((sizeOfSet['neg']-wordExistance['neg'][word])/size_without_word)*numpy.log2((sizeOfSet['neg']-wordExistance['neg'][word])/size_without_word))
        h_C = h_C_with_word + h_C_without_word
        if h_D - h_C > 1:
            output.append(word)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
        
    print("\nComplete selecting words by information gain")
    return output

size = dict()
size = {'pos':len(trainSet['pos']), 'neg':len(trainSet['neg']), 'all':len(trainSet['all'])}
selectedWordSet = selectWords(allWord, advancedNumOfExistance, size)

Start selecting words by information gain
[                    ] 0%

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


[=================== ] 99%
Complete selecting words by information gain


In [18]:
print(len(selectedWordSet), " ", len(allWord))

0   120777


## Task 1 - Bernoulli Naive Bayes

### Task 1.1.1 - Bernoulli Naive Bayes

In [25]:
# @author Pengnan Fan
# @param dataSet: set for prediction
# @param wordExistance: dict {'pos' -> pos word existance, 'neg' -> neg word existance, 'all' -> all word existance}
# @param size: dict {'pos' -> pos size, 'neg' -> neg size, 'all' -> all size}
# @return prediction
def bernoulliNaiveBayes(dataSet, wordExistance, wordSet, size):
    pPos = size['pos']/size['all']
    pNeg = size['neg']/size['all']
    prediction = list()
    count = 0
    lenW = len(dataSet)
    print("Start Bernoulli naive Bayes classifying")
    for exp in dataSet:
        pPos_x = numpy.log([pPos])
        pNeg_x = numpy.log([pNeg])
        
        for word in wordSet:
            pos = 0
            neg = 0
            
            if word in exp:
                pos = (wordExistance['pos'][word]+1)/(size['pos']+2)
                neg = (wordExistance['neg'][word]+1)/(size['neg']+2)
            else:
                pos = 1 - ((wordExistance['pos'][word]+1)/(size['pos']+2))
                neg = 1 - ((wordExistance['neg'][word]+1)/(size['neg']+2))
            
            pPos_x += numpy.log([pos])
            pNeg_x += numpy.log([neg])
            
        log_ratio = pPos_x - pNeg_x
        
        if log_ratio > 0:
            prediction.append(1)
        else:
            prediction.append(0)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    print("\nComplete Bernoulli naive Bayes classifying")
    
    return prediction


#pred = bernoulliNaiveBayes(proceedData_All, advancedNumOfExistance, allWord, size)

### Task 1.1.2 - Bernoulli Naive Bayes with TF * IDF

In [23]:
def bernoulliNaiveBayesWithTFIDF(dataSet, wordExistance, wordSet, size, commentExistances):
    pPos = size['pos']/size['all']
    pNeg = size['neg']/size['all']
    prediction = list()
    count = 0
    lenW = len(dataSet)
    print("Start Bernoulli naive Bayes classifying")
    for i in range(lenW):
        pPos_x = numpy.log([pPos])
        pNeg_x = numpy.log([pNeg])
        
        for word in wordSet:
            pos = 0
            neg = 0
            
            numOfWords = sum(commentExistances[i].values())
            tf = (commentExistances[i][word])/numOfWords
            idf = numpy.log([size['all']/wordExistance['all'][word]])
            
            if word in dataSet[i]:
                pos = ((wordExistance['pos'][word]+1)/(size['pos']+2))*tf*idf
                neg = ((wordExistance['neg'][word]+1)/(size['neg']+2))*tf*idf
            else:
                pos = (1 - ((wordExistance['pos'][word]+1)/(size['pos']+2)))*tf*idf
                neg = (1 - ((wordExistance['neg'][word]+1)/(size['neg']+2)))*tf*idf
            
            pPos_x += numpy.log(list(pos))
            pNeg_x += numpy.log(list(neg))
            
        log_ratio = pPos_x - pNeg_x
        
        if log_ratio > 0:
            prediction.append(1)
        else:
            prediction.append(0)
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(lenW)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    print("\nComplete Bernoulli naive Bayes classifying")
    
    return prediction

advancedPred = bernoulliNaiveBayesWithTFIDF(proceedData_All, advancedNumOfExistance, allWord, size, wordsWithDuplicates['individual'])

In [41]:
print(len(proceedData_All[0]))
print(sum(wordsWithDuplicates['individual'][0].values()))
print(wordsWithDuplicates['individual'][0])

59
65
Counter({'office': 2, 'doubt': 2, 'would': 2, 'conversation': 2, 'film': 2, 'going': 2, 'perhaps': 1, 'arrange': 1, 'meet': 1, 'sitting': 1, 'right': 1, 'facetoface': 1, 'bourne': 1, 'remains': 1, 'street': 1, 'tough': 1, 'elusive': 1, 'inhuman': 1, 'resilience': 1, 'leads': 1, 'little': 1, 'far': 1, 'fantasy': 1, 'macho': 1, 'point': 1, 'shards': 1, 'bond': 1, 'type': 1, 'gallows': 1, 'humour': 1, 'actionbr': 1, 'feeling': 1, 'something': 1, 'another': 1, 'level': 1, 'world': 1, 'live': 1, 'trilogy': 1, 'coveys': 1, 'well': 1, 'scene': 1, 'set': 1, 'waterloo': 1, 'guardian': 1, 'journalist': 1, 'great': 1, 'effect': 1, 'meeting': 1, 'worlds': 1, 'superfluous': 1, 'bodybr': 1, 'shaky': 1, 'annoy': 1, 'much': 1, 'enjoy': 1, 'hope': 1, 'somehow': 1, 'keep': 1, 'franchise': 1})


### Task 1.2 - Evaluation

In [18]:
# @author Pengnan Fan
# @param dataSet: train set with label
# @param prediction: prediction of each example in the trainSet
# @return result: list of results: {TP: true pos, TN: true neg, FP: false pos, FN: false neg}
def evaluation(dataSet, prediction):
    size = len(prediction)
    count = 0
    result = {'TP':0,'TN':0,'FP':0,'FN':0}
    
    print("Start evaluating classification")
    for i in range(size):
        if prediction[i]==1:
            if dataSet[i]['isPos']==1:
                result['TP']+=1
            else:
                result['FP']+=1
        else:
            if dataSet[i]['isPos']==1:
                result['FN']+=1
            else:
                result['TN']+=1
                
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(size)) * 100)
        sys.stdout.write("[%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    print("\nComplete evaluating classification")
    return result

eva = evaluation(trainSet['all'], pred)
print("TP = ", eva['TP'], "\nTN = ", eva['TN'], "\nFP = ", eva['FP'], "\nFN = ", eva['FN'])

Start evaluating classification
[=================== ] 99%
Complete evaluating classification
TP =  11120 
TN =  11982 
FP =  518 
FN =  1380


In [24]:
print("accuracy = ", (eva['TP']+eva['TN'])/250, "%")

accuracy =  92.408 %


### Task 1.3 - Test Set

In [19]:
ADDRESS_TEST_PENGNAN = "D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification\\test"

def loadTestData():
    print("Start loading test set")
    test = sklearn.datasets.load_files("D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification", categories={"test"}, shuffle=False)
    print("Complete loading test set")
    
    testSet = list()
    count = 0
    size = len(test.data)
    for x in test.data:
        testSet.append({"comment":x.decode('utf-8'), "isPos":0})
        
        # This is for loading bar
        sys.stdout.write('\r')
        c = int((float(count) / float(size)) * 100)
        sys.stdout.write("Preparing test set: [%-20s] %d%%" % ('='*int(c / 5), c))
        sleep(0.001)
        sys.stdout.flush()
        count += 1
    
    print()
    print("\nFinish preparing test set")
    return testSet

testSet = loadTestData()

Start loading test set
Complete loading test set
Preparing test set: [=================== ] 99%

Finish preparing test set


In [20]:
editedData = dataStandardization(testSet)

Start proceeding data
[=================== ] 99%
Complete proceeding data


In [21]:
print(len(allWord))

120777


In [26]:
result = bernoulliNaiveBayes(editedData, advancedNumOfExistance, allWord, size)

Start Bernoulli naive Bayes classifying
[=================== ] 99%
Complete Bernoulli naive Bayes classifying


In [27]:
index = []
for i in range(25000):
    index.append(i)
print(index[0:100])
outcome = {"Id":index, "Category": result}

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [28]:
print(outcome['Id'][0:100])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


In [29]:
df = pd.DataFrame(outcome, columns={'Category', 'Id'})

In [30]:
print(df)

          Id  Category
0          0         0
1          1         0
2          2         1
3          3         0
4          4         0
5          5         1
6          6         1
7          7         1
8          8         0
9          9         0
10        10         0
11        11         0
12        12         0
13        13         1
14        14         1
15        15         1
16        16         1
17        17         1
18        18         0
19        19         1
20        20         0
21        21         0
22        22         0
23        23         1
24        24         1
25        25         0
26        26         0
27        27         0
28        28         0
29        29         0
...      ...       ...
24970  24970         1
24971  24971         0
24972  24972         0
24973  24973         1
24974  24974         0
24975  24975         1
24976  24976         0
24977  24977         1
24978  24978         0
24979  24979         1
24980  24980         1
24981  2498

In [31]:
export_csv = df.to_csv ('D:\\McGill\\19Fall\\COMP 551\\Projects\\Project2\\comp-551-imbd-sentiment-classification\\result.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

### Task 1.4 - Cross Validation

In [ ]:
# @author Pengnan Fan
# @param dataSet of comments
def crossValidation(dataSet):
    subSet_pos = list()
    subSet_neg = list()
    a = 0
    b = 0
    
    # It separates the data set into 5 sub sets by pos/neg
    # You may modify it based on your data structure
    for x in range(5):
        a = b
        b += 2500
        subSet_pos.append(dataSet['pos'][a:b])
        subSet_neg.append(dataSet['neg'][a:b])
    
    # It loops 5 times for cross validations
    for i in range(5):
        
        print("<Cross Validation", i, ">")
        posSet = list()
        negSet = list()
        valSet = list()
        cvSet = {}
        
        # It chooses 1 out of 5 subsets as validation set, and learn on the rest
        for j in range(5):
            if j != i:
                # Preparing train set
                posSet += subSet_pos[j]
                negSet += subSet_neg[j]
            else:
                # Preparing validation set
                valSet = subSet_pos[j] + subSet_neg[j]
                
        # -------------------------------------------------------------------- #
        # Add your own proceeding functions and learning functions starts here
        
        prediction = someClassifier(SomeInputs)
        
        # Add your own proceeding functions and learning functions ends here
        # ------------------------------------------------------------------ #
        
        # You may use the evaluation function I wrote here. Please check the input sturcture before use
        result = evaluation(answer, prediction)
        print("True Positive = ", result['TP'], "\nTrue Negative = ", result['TN'], "\nFalse Positive = ", result['FP'], "\nFalse Negative = ", result ['FN'])
        print("Accuracy on validation set ", i, " = ", (result['TP']+result['TF'])/len(valSet))
        
    print("\nComplete cross validation")

crossValidation(trainSet)